# <h1 align="center"> 184.702 Machine Learning - 2022S, Exercise 0
<h5 align="center"> Group 08: Wanecek Wilhelm, Simhandl Stefan, Beck Viktor

In [ ]:
import os
path= ''
currentdirectory = os.path.abspath(path)
os.chdir(currentdirectory)

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# Kinematics-motion-dataset

source:
https://www.kaggle.com/yasserh/kinematics-motion-data

In [ ]:
# import raw data
csvpath = os.path.join("datasets", "kinematics-motion-data.csv")
df=pd.read_csv(csvpath, sep=',', encoding='latin-1')

df.info()
df.head()

## Explore kinematics-motion-dataset

In [ ]:
# explor dataset using pandas profiling
from pandas_profiling import ProfileReport

## drop non predicitve features
# 1) Date, Time and Username are no universal features to predict the activites Running and Walking 
# 2) Further it makes no sense to explore them so we can drop them before exploring the data
df = df.drop(columns=['username','date','time'],axis=1)

prof = ProfileReport(df)#, minimal = True)
prof.to_file(output_file='explore_kinematics-motion-dataset.html')

In [ ]:
df.shape
df.isna().sum()

In [ ]:
# get descriptive statistics
df.describe()

# Communities and Crime dataset

source:
https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime


In [ ]:
# import raw data
csvpath = os.path.join("datasets", 'communities_crime.csv')
input=pd.read_csv(csvpath, sep=';', encoding='latin-1')

input.info()
input.head()

In [ ]:
# get US state postcode mapping from https://github.com/scpike/us-state-county-zip/blob/master/geo-data.csv
csvpath = os.path.join("datasets", 'US_state-fips.csv')
state_fips = pd.read_csv(csvpath, sep=',', encoding='latin-1')

display(state_fips.head(3))
state_fips = state_fips.drop(state_fips.columns[2:6],axis=1)
display(state_fips.head(3))

In [ ]:
# drop fips and add state name
df = input.copy().rename(columns={'state: US state (by number) - not counted as predictive above, but if considered, should be consided nominal (nominal)': 'state_fips',
                                  'communityname: community name - not predictive - for information only (string)': 'communityname'})#, index={'old': 'new'})

## drop non predicitve features
# 1) fold, county, community and communityname are no universal features to predict the crime rate
# 2) Further it makes no sense to explore them so we can drop them before exploring the data
df = df.drop(columns=['fold: fold number for non-random 10 fold cross validation, potentially useful for debugging, paired tests - not predictive (numeric)',
                      'county: numeric code for county - not predictive, and many missing values (numeric)',
                      'community: numeric code for community - not predictive and many missing values (numeric)'],axis=1)

df = df.merge(state_fips, how='left', on='state_fips').drop(columns='state_fips')

df = df.set_index(['state','communityname'])
display(df.head(3))
df.info()
df.shape

## Explore communities and crime dataset

In [ ]:
# explor dataset using pandas profiling
from pandas_profiling import ProfileReport

prof = ProfileReport(df, minimal = True)
prof.to_file(output_file='explore_communities-crime-dataset.html')

In [ ]:
df.shape
df.isna().sum()

In [ ]:
# get descriptive statistics
df.describe()